In [13]:
import os
import sys
sys.path.append(os.path.realpath('../..'))
import aoc
my_aoc = aoc.AdventOfCode(2024,21)

In [14]:
sample_data = {
    "029A": "<vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A",
    "980A": "<v<A>>^AAAvA^A<vA<AA>>^AvAA<^A>A<v<A>A>^AAAvA<^A>A<vA>^A<A>A",
    "179A": "<v<A>>^A<vA<A>>^AAvAA<^A>A<v<A>>^AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A",
    "456A": "<v<A>>^AA<vA<A>>^AAvAA<^A>A<vA>^A<A>A<vA>^A<A>A<v<A>A>^AAvA<^A>A",
    "379A": "<v<A>>^AvA^A<vA<AA>>^AAvA<^A>AAvA^A<vA>^AA<A>A<v<A>A>^AAAvA<^A>A"
}

In [15]:
def calculate_complexity(code: str, sequence: str) -> int:
    # Extract digits from code and convert to integer
    digits = ''.join(char for char in code if char.isdigit())
    # Example complexity: multiply integer value of digits by length of sequence
    complexity = int(digits) * len(sequence)
    print(f"Digits: {digits}, Length: {len(sequence)}, Complexity: {complexity}")
    return complexity



In [16]:
from collections import deque


class KeyPad:
    directions = {'^': (0, 1), 'v': (0, -1), '<': (-1, 0), '>': (1, 0)}
    def __init__(self):
        self.keys = {}
        self.current_key = None
    
    @property
    def position(self):
        return self.keys[self.current_key]
    
    @property
    def key_at_map(self):
        return {v: k for k, v in self.keys.items()}

    def shortest_moves(self, target_key: str) -> str:
        """Return (moves_string, path_keys) from start_key to target_key."""
        if self.current_key == target_key:
            return "", [self.current_key]

        start = self.keys[self.current_key]
        goal  = self.keys[target_key]
        valid = set(self.keys.values())

        q = deque([start])
        parent = {start: None}
        parent_move = {}

        while q:
            x, y = q.popleft()
            if (x, y) == goal:
                break
            for m, (dx, dy) in self.directions.items():
                nx, ny = x + dx, y + dy
                if (nx, ny) in valid and (nx, ny) not in parent:
                    parent[(nx, ny)] = (x, y)
                    parent_move[(nx, ny)] = m
                    q.append((nx, ny))

        # Reconstruct
        if goal not in parent:
            raise ValueError(f"No route from {self.current_key} to {target_key}")

        moves = []
        path_coords = []
        cur = goal
        while cur is not None:
            path_coords.append(cur)
            cur = parent[cur]
        path_coords.reverse()

        # Convert coords to keys and collect moves
        path_keys = [self.key_at_map[c] for c in path_coords]
        for i in range(1, len(path_coords)):
            moves.append(parent_move[path_coords[i]])

        return "".join(moves), path_keys
    
    def get_code_sequence(self, code: str) -> str:
        sequence = ''
        for char in code:
            moves, path = self.shortest_moves(char)
            last = self.current_key
            self.current_key = char
            sequence += moves + 'A'  # 'A' is the press key action
            print(f"Move from {last} to {char}: {moves}")
        return sequence

class DoorKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys =  {
            '7': (0, 3), '8': (1, 3), '9': (2, 3),
            '4': (0, 2), '5': (1, 2), '6': (2, 2),
            '1': (0, 1), '2': (1, 1), '3': (2, 1),
                         '0': (1, 0), 'A': (2, 0),
        }
        self.current_key = 'A'
        self.reset_current = False

class RobotKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys = {
                         '^': (1, 1), 'A': (2, 1),
            '<': (0, 0), 'v': (1, 0), '>': (2, 0)
        }
        self.current_key = 'A'
        self.reset_current = True

In [17]:
door = DoorKeyPad()
robot1 = RobotKeyPad()
robot2 = RobotKeyPad()

code = '029A'
final_sequence = ''
for char in code:
    print(char)
    sequence = door.get_code_sequence(char)
    print(f"First sequence: {sequence}")

    second_sequence = robot1.get_code_sequence(sequence)
    print(f"Second sequence: {second_sequence}")

    third_sequence = robot2.get_code_sequence(second_sequence)
    print(f"Third sequence: {third_sequence}")
    final_sequence += third_sequence

complexity1 = calculate_complexity(code, final_sequence)

print(f"Complexity: {complexity1}")

0
Move from A to 0: <
First sequence: <A
Move from A to <: v<<
Move from < to A: >^>
Second sequence: v<<A>^>A
Move from A to v: v<
Move from v to <: <
Move from < to <: 
Move from < to A: >^>
Move from A to >: v
Move from > to ^: ^<
Move from ^ to >: v>
Move from > to A: ^
Third sequence: v<A<AA>^>AvA^<Av>A^A
2
Move from 0 to 2: ^
First sequence: ^A
Move from A to ^: <
Move from ^ to A: >
Second sequence: <A>A
Move from A to <: v<<
Move from < to A: >^>
Move from A to >: v
Move from > to A: ^
Third sequence: v<<A>^>AvA^A
9
Move from 2 to 9: ^^>
First sequence: ^^>A
Move from A to ^: <
Move from ^ to ^: 
Move from ^ to >: v>
Move from > to A: ^
Second sequence: <AAv>A^A
Move from A to <: v<<
Move from < to A: >^>
Move from A to A: 
Move from A to v: v<
Move from v to >: >
Move from > to A: ^
Move from A to ^: <
Move from ^ to A: >
Third sequence: v<<A>^>AAv<A>A^A<A>A
A
Move from 9 to A: vvv
First sequence: vvvA
Move from A to v: v<
Move from v to v: 
Move from v to v: 
Move from v to A

In [18]:
# <vA<AA>>^AvAA<^A>A<v<A>>^AvA^A<vA>^A<v<A>^A>AAvA^A<v<A>A>^AAAvA<^A>A
# v<A<AA>^>AvA^<Av>A^Av<<A>^>AvA^Av<<A>^>AAv<A>A^A<A>Av<A<A>^>AAA<Av>A^A


# v   <  < A    >  ^   >   A  <    A    >  A  <    A    A v   >  A  ^  A v    <  A    A A ^  >   A
# v<A <A A >^>A vA ^<A v>A ^A v<<A >^>A vA ^A v<<A >^>A A v<A >A ^A <A >A v<A <A >^>A A A <A v>A ^A

In [19]:
from collections import deque
from functools import lru_cache
import heapq


class KeyPad:
    directions = {'^': (0, 1), 'v': (0, -1), '<': (-1, 0), '>': (1, 0)}

    def __init__(self):
        self.keys = {}
        self.current_key = None  # pointer on THIS keypad

    @property
    def pos_of(self):
        return self.keys

    @property
    def key_at(self):
        # inverse map (computed on demand; layout is tiny)
        return {v: k for k, v in self.keys.items()}

    def _neighbors(self, key):
        """Yield (dir_symbol, neighbor_key) from key on this keypad."""
        x, y = self.pos_of[key]
        inv = self.key_at
        # Stable order = stable tie-breaking; tweak if you want.
        for d in ('<', '>', '^', 'v'):
            dx, dy = self.directions[d]
            np = (x + dx, y + dy)
            if np in inv:
                yield d, inv[np]

    def _bfs_unweighted_moves(self, start_key, target_key):
        """Shortest unweighted path (as string of directions) on THIS pad."""
        if start_key == target_key:
            return ""
        inv = self.key_at
        start = self.pos_of[start_key]
        goal = self.pos_of[target_key]
        q = deque([start])
        parent = {start: None}
        parent_move = {}
        while q:
            xy = q.popleft()
            if xy == goal:
                break
            k_here = inv[xy]
            for d, k_next in self._neighbors(k_here):
                nxt = self.pos_of[k_next]
                if nxt not in parent:
                    parent[nxt] = xy
                    parent_move[nxt] = d
                    q.append(nxt)
        if goal not in parent:
            raise ValueError(f"No route from {start_key} to {target_key}")
        # reconstruct
        moves = []
        cur = goal
        while parent[cur] is not None:
            moves.append(parent_move[cur])
            cur = parent[cur]
        return "".join(reversed(moves))

    # ---------- ROBUST ENCODER PRIMITIVES ----------

    def _dijkstra_move_cost_and_seq(self, start_key, target_key, controller_pad, depth, controller_start_cursor):
        """
        Move pointer on THIS keypad from start_key to target_key.
        Costs are the true cost of making the controller pad press directions.
        Returns (emitted_seq_on_human, controller_end_cursor, total_cost).
        """
        if start_key == target_key:
            return "", controller_start_cursor, 0

        # State carries where the CONTROLLER's pointer is
        start_state = (start_key, controller_start_cursor)
        goal_key = target_key

        pq = [(0, start_state)]
        best = {start_state: 0}
        parent = {}  # child_state -> (prev_state, seq_emitted_for_edge)

        while pq:
            cost, (here_key, ctrl_cur) = heapq.heappop(pq)
            if cost != best.get((here_key, ctrl_cur)):
                continue
            if here_key == goal_key:
                end_state = (here_key, ctrl_cur)
                break

            for d, nxt_key in self._neighbors(here_key):
                # cost to press direction d on controller, starting from ctrl_cur
                seq_edge, edge_cost, ctrl_end = controller_pad._select_and_press(ctrl_cur, d, depth - 1)
                nxt_state = (nxt_key, ctrl_end)   # controller now sits on that key (usually d)
                new_cost = cost + edge_cost
                if new_cost < best.get(nxt_state, float('inf')):
                    best[nxt_state] = new_cost
                    parent[nxt_state] = ((here_key, ctrl_cur), seq_edge)
                    heapq.heappush(pq, (new_cost, nxt_state))
        else:
            raise ValueError(f"No path from {start_key} to {target_key}")

        # reconstruct emitted sequence and return controller end cursor
        seq_parts = []
        s = end_state
        while s != start_state:
            prev, seq_edge = parent[s]
            seq_parts.append(seq_edge)
            s = prev
        seq_parts.reverse()
        emitted = "".join(seq_parts)
        _, controller_end_cursor = end_state
        return emitted, controller_end_cursor, best[end_state]

    def encode_code(self, code: str, controller_pad, depth: int, verbose=False) -> str:
        seq = []
        cur = self.current_key
        ctrl_cursor = 'A'  # controller starts on 'A' between digits
        for ch in code:
            # 1) move THIS pointer to ch
            move_seq, ctrl_after_move, _ = self._dijkstra_move_cost_and_seq(
                cur, ch, controller_pad, depth, controller_start_cursor=ctrl_cursor
            )
            if verbose:
                print(f"Move from {cur} to {ch}: {move_seq}")
            seq.append(move_seq)

            # 2) press A on controller to activate ch on THIS keypad
            press_seq, _, ctrl_after_press = controller_pad._select_and_press(ctrl_after_move, 'A', depth - 1)
            if verbose:
                print(f"Press on controller: {press_seq}")
            seq.append(press_seq)

            cur = ch
            ctrl_cursor = ctrl_after_press  # will be 'A' after a press at any depth > 0
        self.current_key = cur
        return "".join(seq)


class DoorKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys = {
            '7': (0, 3), '8': (1, 3), '9': (2, 3),
            '4': (0, 2), '5': (1, 2), '6': (2, 2),
            '1': (0, 1), '2': (1, 1), '3': (2, 1),
                          '0': (1, 0), 'A': (2, 0),
        }
        self.current_key = 'A'


class RobotKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys = {
                          '^': (1, 1), 'A': (2, 1),
            '<': (0, 0),   'v': (1, 0), '>': (2, 0),
        }
        self.current_key = 'A'

    # LRU cache over the wrapper to memoize by (start_on_self, symbol, depth)
    @lru_cache(maxsize=None)
    def _select_and_press(self, start_on_self: str, symbol: str, depth: int):
        """
        On THIS robot keypad, starting with pointer at `start_on_self`,
        produce the minimal human sequence to select `symbol` and then 'press it'
        (i.e., press A on the pad BELOW). There are `depth` more robot layers beneath this one.

        Returns: (emitted_seq_on_human, total_cost, end_cursor_on_THIS_pad)
        """
        return self._select_and_press_impl(start_on_self, symbol, depth)

    def _select_and_press_impl(self, start_on_self: str, symbol: str, depth: int):
        # ---- BASE: human presses directly on THIS pad ----
        if depth == 0:
            # Human moves to `symbol` and then presses A on THIS pad.
            path = self._bfs_unweighted_moves(start_on_self, symbol)  # e.g., "v<<"
            full_seq = path + 'A'                                     # e.g., "v<<A"
            end_on_self = 'A'                                         # pointer ends on 'A' (last press)
            return full_seq, len(full_seq), end_on_self

        # ---- RECURSIVE: deeper robot controls THIS pad ----
        # Step 1: move THIS pointer to `symbol`, with the deeper controller starting at 'A'
        move_seq, lower_ctrl_after_move, _ = self._dijkstra_move_cost_and_seq(
            start_on_self,
            symbol,
            controller_pad=self,             # the lower layer has the same layout
            depth=depth,
            controller_start_cursor='A'      # after any completed press, lower cursor is 'A'
        )

        # Step 2: deeper controller presses its own 'A' to activate `symbol` here
        press_seq, _, lower_ctrl_after_press = self._select_and_press(lower_ctrl_after_move, 'A', depth - 1)
        # lower_ctrl_after_press will be 'A' again

        full_seq = move_seq + press_seq
        end_on_self = symbol                  # THIS pad stays on the symbol; the press happened below
        return full_seq, len(full_seq), end_on_self



In [20]:
# -------------------------
# Example usage (Part 1)
# -------------------------

door = DoorKeyPad()
robot = RobotKeyPad()

code = '029A'

# Two robot pads under the door for Part 1:
# (robot1 controls the door; robot2 controls robot1; human controls robot2)
depth_under_door = 2

final_sequence = door.encode_code(code, controller_pad=robot, depth=depth_under_door, verbose=True)

print(f"Digits: {code}, Length: {len(final_sequence)}")
# If you have your existing function:
complexity1 = calculate_complexity(code, final_sequence)
print(f"Complexity: {complexity1}")

Move from A to 0: <vA<v<A<v<AA
Press on controller: vAvA<AA
Move from 0 to 2: <v<AA
Press on controller: vAA
Move from 2 to 9: <v<AAAvA<vAA
Press on controller: <AA
Move from 9 to A: <vA<v<AAAA
Press on controller: vA<AA
Digits: 029A, Length: 57
Digits: 029, Length: 57, Complexity: 1653
Complexity: 1653


In [21]:
from collections import deque, defaultdict
from functools import lru_cache
import heapq


class KeyPad:
    directions = {'^': (0, 1), 'v': (0, -1), '<': (-1, 0), '>': (1, 0)}

    def __init__(self):
        self.keys = {}
        self.current_key = None

    @property
    def pos_of(self):
        return self.keys

    @property
    def key_at(self):
        return {v: k for k, v in self.keys.items()}

    def _neighbors(self, key):
        x, y = self.pos_of[key]
        inv = self.key_at
        # Stable order keeps outputs deterministic (affects tie-breaking only)
        for d in ('<', '>', '^', 'v'):
            dx, dy = self.directions[d]
            np = (x + dx, y + dy)
            if np in inv:
                yield d, inv[np]

    def _bfs_unweighted_moves(self, start_key, target_key):
        """One shortest path as a directions string on THIS pad."""
        if start_key == target_key:
            return ""
        inv = self.key_at
        start = self.pos_of[start_key]
        goal = self.pos_of[target_key]
        q = deque([start])
        parent = {start: None}
        parent_move = {}
        while q:
            xy = q.popleft()
            if xy == goal:
                break
            k_here = inv[xy]
            for d, k_next in self._neighbors(k_here):
                nxt = self.pos_of[k_next]
                if nxt not in parent:
                    parent[nxt] = xy
                    parent_move[nxt] = d
                    q.append(nxt)
        if goal not in parent:
            raise ValueError(f"No route from {start_key} to {target_key}")
        # reconstruct
        moves = []
        cur = goal
        while parent[cur] is not None:
            moves.append(parent_move[cur])
            cur = parent[cur]
        return "".join(reversed(moves))

    def _all_shortest_paths(self, start_key, target_key):
        """
        Return ALL shortest paths (strings of ^v<>) from start_key to target_key
        on THIS pad.
        """
        if start_key == target_key:
            return [""]

        inv = self.key_at
        start = self.pos_of[start_key]
        goal = self.pos_of[target_key]

        # 1) BFS to compute distance to each reachable cell
        dist = {start: 0}
        q = deque([start])
        while q:
            xy = q.popleft()
            if xy == goal:
                # we don't break early; we still want correct dist for backtracking,
                # but early break is okay too—leave it if you prefer
                pass
            k_here = inv[xy]
            for d, k_next in self._neighbors(k_here):
                nxt = self.pos_of[k_next]
                if nxt not in dist:
                    dist[nxt] = dist[xy] + 1
                    q.append(nxt)

        if goal not in dist:
            raise ValueError(f"No route from {start_key} to {target_key}")

        # 2) DFS backtracking via predecessors; append the FORWARD move
        paths = []
        order = ('<', '>', '^', 'v')  # stable tie-breaking
        vec = self.directions

        def dfs(xy, acc):
            if xy == start:
                paths.append("".join(reversed(acc)))
                return
            x, y = xy
            for d in order:
                dx, dy = vec[d]
                prev = (x - dx, y - dy)  # predecessor that would step forward by 'd' to reach xy
                if prev in dist and dist[prev] == dist[xy] - 1:
                    dfs(prev, acc + [d])

        dfs(goal, [])
        # unique + deterministic order
        return sorted(set(paths))

    # --------- ROBUST MOVE via controller over all shortest paths ----------

    def _move_via_controller_over_shortest_paths(self, start_key, target_key, controller_pad, depth, controller_start_cursor):
        """
        Move THIS pad's pointer from start_key to target_key by trying all shortest
        direction strings, simulating the TRUE cost on the controller pad below.

        Returns (emitted_seq_on_human, controller_end_cursor, total_cost)
        where controller_* refer to the pad immediately below THIS one.
        """
        if start_key == target_key:
            return "", controller_start_cursor, 0

        best_cost = float('inf')
        best_seq = None
        best_cursor = None

        for path in self._all_shortest_paths(start_key, target_key):
            cur_cursor = controller_start_cursor
            seq_parts = []
            cost = 0
            for d in path:
                # Each single direction press is realized by "select&press d" on the controller
                sub_seq, sub_cost, cur_cursor = controller_pad._select_and_press(cur_cursor, d, depth - 1)
                seq_parts.append(sub_seq)
                cost += sub_cost
                if cost >= best_cost:
                    break  # prune
            else:
                if cost < best_cost:
                    best_cost = cost
                    best_seq = "".join(seq_parts)
                    best_cursor = cur_cursor

        if best_seq is None:
            raise ValueError(f"No path from {start_key} to {target_key}")
        return best_seq, best_cursor, best_cost

    def encode_code(self, code: str, controller_pad, depth: int, verbose=False) -> str:
        """
        Encode `code` on THIS keypad, using `controller_pad` below with `depth`
        direction layers beneath THIS pad.
        """
        seq = []
        cur = self.current_key
        ctrl_cursor = 'A'  # controller starts on 'A' between *digits*
        for ch in code:
            # 1) move THIS pointer to ch (try all shortest paths; minimize true cost)
            move_seq, ctrl_after_move, _ = self._move_via_controller_over_shortest_paths(
                cur, ch, controller_pad, depth, controller_start_cursor=ctrl_cursor
            )
            if verbose:
                print(f"Move from {cur} to {ch}: {move_seq}")
            seq.append(move_seq)

            # 2) press 'A' on the controller to activate ch on THIS keypad
            press_seq, _, ctrl_after_press = controller_pad._select_and_press(ctrl_after_move, 'A', depth - 1)
            if verbose:
                print(f"Press on controller: {press_seq}")
            seq.append(press_seq)

            cur = ch
            ctrl_cursor = ctrl_after_press  # will end on 'A'
        self.current_key = cur
        return "".join(seq)


class DoorKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys = {
            '7': (0, 3), '8': (1, 3), '9': (2, 3),
            '4': (0, 2), '5': (1, 2), '6': (2, 2),
            '1': (0, 1), '2': (1, 1), '3': (2, 1),
                          '0': (1, 0), 'A': (2, 0),
        }
        self.current_key = 'A'


class RobotKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys = {
                          '^': (1, 1), 'A': (2, 1),
            '<': (0, 0),   'v': (1, 0), '>': (2, 0),
        }
        self.current_key = 'A'

    @lru_cache(maxsize=None)
    def _select_and_press(self, start_on_self: str, symbol: str, depth: int):
        """
        On THIS direction pad, starting with pointer at `start_on_self`,
        emit the minimal human sequence to 'select and press' `symbol`.
        If depth==0, human is pressing here; otherwise a deeper robot is.
        Returns (emitted_seq_on_human, total_cost, end_cursor_on_THIS_pad).
        """
        return self._select_and_press_impl(start_on_self, symbol, depth)

    def _select_and_press_impl(self, start_on_self: str, symbol: str, depth: int):
        # ---- BASE: human presses directly on THIS pad ----
        if depth == 0:
            # Human moves from start_on_self -> symbol (on THIS pad), then presses A (on THIS pad)
            # Each keypress costs 1.
            path = self._bfs_unweighted_moves(start_on_self, symbol)
            full_seq = path + 'A'
            end_on_self = 'A'  # last physical press is 'A'
            return full_seq, len(full_seq), end_on_self

        # ---- RECURSIVE: a deeper robot controls THIS pad ----
        # Step 1: move THIS pointer to `symbol` using the deeper controller,
        # minimizing TRUE cost across all shortest paths.
        move_seq, lower_cursor_after_move, _ = self._move_via_controller_over_shortest_paths(
            start_on_self, symbol, controller_pad=self, depth=depth, controller_start_cursor='A'
        )

        # Step 2: press 'A' on the deeper controller to activate `symbol` here.
        press_seq, _, _ = self._select_and_press(lower_cursor_after_move, 'A', depth - 1)

        full_seq = move_seq + press_seq
        end_on_self = symbol  # THIS pad stays on the selected symbol; the 'A' was pressed below.
        return full_seq, len(full_seq), end_on_self
4

4

In [22]:
# Robot pad sanity:
rp = RobotKeyPad()
assert rp._bfs_unweighted_moves('A', '<') in {'<v<', 'v<<'}  # depending on neighbor order
assert '<v<' in rp._all_shortest_paths('A', '<') or 'v<<' in rp._all_shortest_paths('A', '<')

# Door pad sanity:
dp = DoorKeyPad()
assert dp._bfs_unweighted_moves('A', '0') == '<'  # one step left is correct


door = DoorKeyPad()
robot = RobotKeyPad()

code = '029A'
depth_under_door = 2   # two robots below the door for Part 1

final_sequence = door.encode_code(code, controller_pad=robot, depth=depth_under_door, verbose=True)
print(f"Digits: {code}, Length: {len(final_sequence)}")
complexity1 = calculate_complexity(code, final_sequence)
print(f"Complexity: {complexity1}")


Move from A to 0: <v<A<vA<v<AA
Press on controller: vAvA<AA
Move from 0 to 2: <v<AA
Press on controller: vAA
Move from 2 to 9: <vAA<v<A<AAA
Press on controller: vAA
Move from 9 to A: <v<A<vAAAA
Press on controller: vA<AA
Digits: 029A, Length: 57
Digits: 029, Length: 57, Complexity: 1653
Complexity: 1653


In [23]:
from collections import deque
from functools import lru_cache

class KeyPad:
    directions = {'^': (0, 1), 'v': (0, -1), '<': (-1, 0), '>': (1, 0)}

    def __init__(self):
        self.keys = {}
        self.current_key = None

    @property
    def pos_of(self):
        return self.keys

    @property
    def key_at(self):
        return {v: k for k, v in self.keys.items()}

    def _neighbors(self, key):
        x, y = self.pos_of[key]
        inv = self.key_at
        # Stable order only affects tie-strings, not minimal length
        for d in ('<', '>', '^', 'v'):
            dx, dy = self.directions[d]
            np = (x + dx, y + dy)
            if np in inv:
                yield d, inv[np]

    def _bfs_unweighted_moves(self, start_key, target_key):
        if start_key == target_key:
            return ""
        inv = self.key_at
        start = self.pos_of[start_key]
        goal = self.pos_of[target_key]
        q = deque([start])
        parent = {start: None}
        parent_move = {}
        while q:
            xy = q.popleft()
            if xy == goal:
                break
            k_here = inv[xy]
            for d, k_next in self._neighbors(k_here):
                nxt = self.pos_of[k_next]
                if nxt not in parent:
                    parent[nxt] = xy
                    parent_move[nxt] = d
                    q.append(nxt)
        if goal not in parent:
            raise ValueError(f"No route from {start_key} to {target_key}")
        moves = []
        cur = goal
        while parent[cur] is not None:
            moves.append(parent_move[cur])
            cur = parent[cur]
        return "".join(reversed(moves))

    def _all_shortest_paths(self, start_key, target_key):
        if start_key == target_key:
            return [""]

        inv = self.key_at
        start = self.pos_of[start_key]
        goal = self.pos_of[target_key]

        # distance map
        dist = {start: 0}
        q = deque([start])
        while q:
            xy = q.popleft()
            k_here = inv[xy]
            for d, k_next in self._neighbors(k_here):
                nxt = self.pos_of[k_next]
                if nxt not in dist:
                    dist[nxt] = dist[xy] + 1
                    q.append(nxt)

        if goal not in dist:
            raise ValueError(f"No route from {start_key} to {target_key}")

        # backtrack all shortest paths by predecessors
        paths = []
        order = ('<', '>', '^', 'v')
        vec = self.directions

        def dfs(xy, acc):
            if xy == start:
                paths.append("".join(reversed(acc)))
                return
            x, y = xy
            for d in order:
                dx, dy = vec[d]
                prev = (x - dx, y - dy)
                if prev in dist and dist[prev] == dist[xy] - 1:
                    dfs(prev, acc + [d])

        dfs(goal, [])
        return sorted(set(paths))

    # --------- robust move: try all shortest paths; simulate controller cost ----------
    def _move_via_controller(self, start_key, target_key, controller_pad, depth, controller_start_cursor):
        if start_key == target_key:
            return "", controller_start_cursor, 0

        best_cost = float('inf')
        best_seq = None
        best_cursor = None

        for path in self._all_shortest_paths(start_key, target_key):
            cur_cursor = controller_start_cursor
            seq_parts = []
            cost = 0
            ok = True
            for d in path:
                sub_seq, sub_cost, cur_cursor = controller_pad._select_and_press(cur_cursor, d, depth - 1)
                seq_parts.append(sub_seq)
                cost += sub_cost
                if cost >= best_cost:
                    ok = False
                    break
            if ok and cost < best_cost:
                best_cost = cost
                best_seq = "".join(seq_parts)
                best_cursor = cur_cursor

        return best_seq, best_cursor, best_cost

    def encode_code(self, code: str, controller_pad, depth: int, verbose=False) -> str:
        seq = []
        cur = self.current_key
        ctrl_cursor = 'A'  # controller starts on A between digits
        for ch in code:
            # move THIS pointer to target key using true controller cost
            move_seq, ctrl_cursor, _ = self._move_via_controller(
                cur, ch, controller_pad, depth, controller_start_cursor=ctrl_cursor
            )
            if verbose:
                print(f"Move from {cur} to {ch}: {move_seq}")
            seq.append(move_seq)

            # press A on controller to activate ch here
            press_seq, _, ctrl_cursor = controller_pad._select_and_press(ctrl_cursor, 'A', depth - 1)
            if verbose:
                print(f"Press on controller: {press_seq}")
            seq.append(press_seq)

            # after a completed press, controller cursor must be back on 'A'
            assert ctrl_cursor == 'A', "controller must be on 'A' after a completed press"
            assert ctrl_cursor == 'A'
            cur = ch

        self.current_key = cur
        return "".join(seq)


class DoorKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys = {
            '7': (0, 3), '8': (1, 3), '9': (2, 3),
            '4': (0, 2), '5': (1, 2), '6': (2, 2),
            '1': (0, 1), '2': (1, 1), '3': (2, 1),
                          '0': (1, 0), 'A': (2, 0),
        }
        self.current_key = 'A'


class RobotKeyPad(KeyPad):
    def __init__(self):
        super().__init__()
        self.keys = {
                          '^': (1, 1), 'A': (2, 1),
            '<': (0, 0),   'v': (1, 0), '>': (2, 0),
        }
        self.current_key = 'A'

    @lru_cache(maxsize=None)
    def _select_and_press(self, start_on_self: str, symbol: str, depth: int):
        return self._select_and_press_impl(start_on_self, symbol, depth)

    def _select_and_press_impl(self, start_on_self: str, symbol: str, depth: int):
        # ---- BASE: human presses directly on THIS pad ----
        if depth == 0:
            # Move THIS pad's cursor to `symbol`, then press A on THIS pad.
            path = self._bfs_unweighted_moves(start_on_self, symbol)  # e.g. "<v<"
            full_seq = path + 'A'                                     # e.g. "<v<A"
            end_on_self = symbol                                      # pressing A doesn't move the cursor
            return full_seq, len(full_seq), end_on_self

        # ---- RECURSIVE: deeper robot controls THIS pad ----
        move_seq, lower_cursor, _ = self._move_via_controller(
            start_on_self, symbol, controller_pad=self, depth=depth, controller_start_cursor='A'
        )
        press_seq, _, lower_cursor = self._select_and_press(lower_cursor, 'A', depth - 1)

        full_seq = move_seq + press_seq
        # After a completed press, the LOWER pad is back on 'A'; THIS pad remains on the selected symbol.
        return full_seq, len(full_seq), symbol

In [24]:
# Robot pad: A -> '<' shortest paths
rp = RobotKeyPad()
assert rp._bfs_unweighted_moves('A','<') in {'<v<','v<<'}
paths = rp._all_shortest_paths('A','<')
assert '<v<' in paths or 'v<<' in paths

# Door pad: A -> 0 is one step left
dp = DoorKeyPad()
assert dp._bfs_unweighted_moves('A','0') == '<'


In [31]:
complexity = 0
for code in sample_data.keys():
    door = DoorKeyPad()
    robot = RobotKeyPad()

    depth_under_door = 2

    final_sequence = door.encode_code(code, controller_pad=robot, depth=depth_under_door, verbose=False)
    # print(f"Digits: {code}, Length: {len(final_sequence)}")
    complexity += calculate_complexity(code, final_sequence)
print(f"Complexity: {complexity}")


Digits: 029, Length: 68, Complexity: 1972
Digits: 980, Length: 60, Complexity: 58800
Digits: 179, Length: 68, Complexity: 12172
Digits: 456, Length: 64, Complexity: 29184
Digits: 379, Length: 64, Complexity: 24256
Complexity: 126384


In [26]:
# inside KeyPad.encode_code loop, after pressing:

# inside RobotKeyPad._select_and_press (base):
